In [60]:
import pandas as pd
import json
import re
import snowflake.connector
from pandas.io.json import json_normalize

#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_PROD',
                                 role='GENERAL_RO')

cs = con.cursor()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [61]:
#Get jason data from snowflake
sql = f'''
Select
  subtask,response
from
  taskattempts
where
  project = '640a1b2121f00794ddf64b06'
  and review_level = 12
  and cast(attempted_at as Date) > '2023-04-30'
limit
  100
'''
cs.execute(sql)
tdf = cs.fetch_pandas_all()

In [62]:
json_str = tdf['RESPONSE'][0]


def flatten_json(nested_json, parent_key='', sep=' '):
    items = []
    for key, value in nested_json.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key
        if isinstance(value, dict):
            items.extend(flatten_json(value, new_key, sep=sep).items())
        elif isinstance(value, list):
            for index, item in enumerate(value):
                sub_key = f"{new_key}{sep}{index}"
                if isinstance(item, dict):
                    items.extend(flatten_json(item, sub_key, sep=sep).items())
                else:
                    items.append((sub_key, item))
        else:
            items.append((new_key, value))
    return dict(items)

xdf = flatten_json(json_str)


AttributeError: 'str' object has no attribute 'items'

In [64]:
print(json_str)

{
  "annotations": {
    "form_character": {
      "field_id": "form_character",
      "response": [
        {
          "character": {
            "field_id": "character",
            "response": [
              "Caparienne Bigfoot"
            ],
            "type": "text"
          },
          "form_character__cannot_determine": {
            "field_id": "form_character__cannot_determine",
            "response": [
              false
            ],
            "type": "boolean"
          },
          "form_character__optional": {
            "field_id": "form_character__optional",
            "response": [
              ""
            ],
            "type": "text"
          }
        }
      ],
      "type": "field_set"
    },
    "form_color": {
      "field_id": "form_color",
      "response": [
        {
          "color": {
            "field_id": "color",
            "response": [
              "Hot Pink"
            ],
            "type": "text"
          },
          "form_

In [ ]:
print(df)
#save the dataframe to a csv file
df.to_csv('test.csv', index=False)